In [ ]:
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2, InceptionV3, DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras import mixed_precision, regularizers
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from imutils import paths
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import cv2
import os
import multiprocessing
mixed_precision.set_global_policy('mixed_float16')
from tensorflow.keras.callbacks import EarlyStopping

# Download dataset

In [ ]:
path = kagglehub.dataset_download("tawsifurrahman/covid19-radiography-database")
print("Path to dataset files:", path)

In [ ]:
dataset_dir =
"/kaggle/input/covid19-radiography-database/COVID-
19_Radiography_Dataset"
image_folders = {
"COVID": os.path.join(dataset_dir,"COVID/images"),
"Lung_Opacity": os.path.join(dataset_dir,"Lung_Opacity/images"),
"Normal": os.path.join(dataset_dir,"Normal/images"),
"Viral Pneumonia": os.path.join(dataset_dir,"Viral Pneumonia/images"),
}
output_dir ="processed_dataset"
os.makedirs(output_dir, exist_ok=True)
def copy_images(category,src_path):
  dest_path = os.path.join(output_dir, category)
  os.makedirs(dest_path, exist_ok=True)
  for img in os.listdir(src_path):
    shutil.copy(os.path.join(src_path, img), os.path.join(dest_path, img))

for category, path in image_folders.items():
  os.makedirs(f"processed_dataset/{category}", exist_ok=True)
  for img in os.listdir(path):
    shutil.copy(os.path.join(path, img),f"processed_dataset/{category}/{img}")
with multiprocessing.Pool(processes=4) as pool:
  pool.starmap(copy_images, image_folders.items())

# **Data Augmentation and Data slpit**

In [ ]:
img_size = (224,224)
batch_size =32
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
validation_split=0.2
)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
"processed_dataset",
target_size=img_size,
batch_size=batch_size,
class_mode="categorical",
subset="training",
shuffle=True
)
val_generator = val_datagen.flow_from_directory(
"processed_dataset",
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
subset="validation",
shuffle=True
)


In [2]:
def build_model(base_model_class, input_shape=(224,224,3),num_classes=4,dropout_rate=0.5,regularization_rate=0.02):
  base_model = base_model_class(input_shape=input_shape, include_top=False, weights='imagenet', pooling='max')
  base_model.trainable =True
  for layer in base_model.layers[:-20]:
    layer.trainable =False
  model = Sequential([base_model,
  BatchNormalization(),
  Dense(256, activation='relu', kernel_regularizer=regularizers.l2(regularization_rate)),
  Dropout(dropout_rate),
  Dense(num_classes, activation='softmax')
  ])
  return model

In [ ]:
def train_model(model,train_generator,val_generator,learning_rate=0.001,epochs=15,class_weights=None,patience=3):
  model.compile(optimizer=Adamax(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy','recall'])
  early_stopping = EarlyStopping(monitor='val_loss',# stops when validation loss stops improving
  patience=patience,# number of epochs to wait before stopping
  restore_best_weights=True)
  history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    verbose=1,
    class_weight=class_weights,
    callbacks=[early_stopping])
  return model, history



def evaluate_model(model,val_generator):
  y_true = np.array([val_generator.classes[i] for i in val_generator.index_array])
  y_pred_probs = model.predict(val_generator)
  y_pred = np.argmax(y_pred_probs, axis=1)
  cm = confusion_matrix(y_true, y_pred)
  plt.figure(figsize=(8,6))
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=val_generator.class_indices.keys(),yticklabels=val_generator.class_indices.keys())
  plt.xlabel("Predicted Label")
  plt.ylabel("True Label")
  plt.title("Confusion Matrix")
  plt.show()
  print(classification_report(y_true, y_pred, target_names=val_generator.class_indices.keys()))
  return y_pred_probs

def plot_metrics(history,y_pred_probs,val_generator,num_classes=4):
  epochs_range =range(1,len(history.history['accuracy']) +1)
  plt.figure(figsize=(8,5))
  plt.plot(epochs_range, history.history['accuracy'], label='Training Accuracy', linestyle='-', marker='o')
  plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy', linestyle='--', marker='o')
  plt.plot(epochs_range, history.history['recall'], label='Training Recall', linestyle='-', marker='s')
  plt.plot(epochs_range, history.history['val_recall'], label='Validation Recall', linestyle='--', marker='s')
  plt.xlabel('Epoch')
  plt.ylabel('Metric Value')
  plt.legend()
  plt.title('Accuracy & Recall')
  plt.show()
  plt.figure(figsize=(8,5))
  plt.plot(epochs_range, history.history['loss'], label= 'Training Loss', linestyle='-', marker='o')
  plt.plot(epochs_range, history.history[ 'val_loss'], label='Validation Loss', linestyle='--', marker='o')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.title('Model Loss')
  plt.show()

# **CHEXNET MODEL**

In [ ]:
# Parameters
input_shape = (224,224,3)
num_classes =4
learning_rate =0.001
epochs =10
dropout_rate =0.5
regularization_rate =0
# Build model
model = build_model(DenseNet121, input_shape, num_classes, dropout_rate, regularization_rate)
# Train model
model, history = train_model(model, train_generator, val_generator, learning_rate, epochs)
# Evaluate model
y_pred_probs = evaluate_model(model, val_generator)
# Plot results
plot_metrics(history, y_pred_probs, val_generator,num_classes)